In [ ]:
import pandas as pd
import numpy as ny

In [ ]:
import requests
url = 'https://drive.google.com/uc?export=download&id=1YsFomiNyVfWYSEZPZ1c8gzXWMwI9PFZj'
url2 = 'https://drive.google.com/uc?export=download&id=1T-pm1OUQfUaiIULrpa_Zz07v5LL63mII'
filename = 'tmdb_5000_movies.csv'
filename2 = 'tmdb_5000_credits.csv'
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"File '{filename}' has been downloaded.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")
  
response = requests.get(url2)

if response.status_code == 200:
    with open(filename2, 'wb') as file:
        file.write(response.content)
    print(f"File '{filename2}' has been downloaded.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

In [ ]:
movies= pd.read_csv('tmdb_5000_movies.csv')
credits= pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(2)


In [ ]:
credits.head()

In [ ]:
credits.head()['crew'].values


In [ ]:
movies=movies.merge(credits,on='title')


In [ ]:
movies.head(1)

In [ ]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()


In [ ]:
movies.duplicated().sum()

In [ ]:
movies['genres']

In [ ]:
import ast 

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies['cast'][0]

In [ ]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter +=1
        else:
            break

    return L
         
        

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies['cast']


In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']== 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew']= movies['crew'].apply(fetch_director)

In [ ]:
movies['crew']

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags']= movies['overview']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
movies['tags'][0]

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

In [ ]:
new_df['tags'].head()

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors= cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors


In [ ]:
vectors.shape

In [ ]:
vectors[0]

In [ ]:
vectors


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
 cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse = True, key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)
    

In [ ]:
recommend('Batman Begins')